In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import matplotlib
import seaborn as sns

# 우선순위 분석 대상 변수

|  변수명  |  설명  |  우선순위  |  이유  |
|---------------|--------------------------------------|------------------------------------------------------|------------------------------------------------------|
|  sum_click  |  학생의 클릭 횟수 (학생의 활동 정도)  |  🔥 높은 우선순위  |  클릭 횟수는 학생이 얼마나 학습에 참여하고 있는지를 보여줍니다. 이탈 예측에 중요한 변수입니다.  |
|  id_student  |  학생 ID  |  🔥 높은 우선순위  |  학생별로 이탈 여부를 추적할 수 있는 주요 식별자입니다. 학생의 학습 패턴과 연관된 분석을 위해 필요해요.  |
|  date  |  활동 일자  |  🟠 중간 우선순위  |  학생의 활동 시점을 알 수 있어요. 시간대별, 주간별 활동 변화를 분석할 수 있기 때문에 학습 패턴 분석에 유용합니다.  |
|  code_module  |  강좌 코드 (학생이 수강한 강좌)  |  🟠 중간 우선순위  |  어떤 강좌를 수강했는지 알 수 있어, 이탈 패턴이 강좌별로 다른지 분석할 수 있어요.  |
|  code_presentation  |  강좌 제공 학기 (학생이 수강한 학기)  |  🟡 낮은 우선순위  |  특정 학기에 따라 이탈 패턴이 달라질 수 있지만, 이 변수는 이탈 예측에 상대적으로 적은 영향을 미칠 수 있어요.  |
|  id_site  |  학생이 활동한 사이트 ID  |  🟡 낮은 우선순위  |  사이트에 따른 활동 차이가 있을 수 있지만, 이탈 예측에 있어서 상대적으로 덜 중요한 변수일 수 있어요.  |


In [2]:
df = pd.read_csv('./data/studentVle.csv')
df

,code_module,code_presentation,id_student,id_site,date,sum_click
0,AAA,2013J,28400,546652,-10,4
1,AAA,2013J,28400,546652,-10,1
2,AAA,2013J,28400,546652,-10,1
3,AAA,2013J,28400,546614,-10,11
4,AAA,2013J,28400,546714,-10,1
...,...,...,...,...,...,...
10655275,GGG,2014J,675811,896943,269,3
10655276,GGG,2014J,675578,896943,269,1
10655277,GGG,2014J,654064,896943,269,3
10655278,GGG,2014J,654064,896939,269,1


# 이탈 예측
<br>

### 우선순위 1: 핵심 컬럼 (가장 중요한 컬럼)

| 컬럼명          | 설명                                   | 이유                                                   |
|---------------|--------------------------------------|------------------------------------------------------|
|id_student|학습자 ID|학습자별로 활동을 추적할 수 있어. 이탈을 예측하려면 개별 학습자의 행동 패턴을 파악하는 것이 중요.|
|date|강의 시작 후 며칠째인지| 시간에 따른 학습자의 활동 변화를 분석할 수 있어. 특정 시점에서 활동이 줄어들면 이탈 가능성이 높다고 볼 수 있음.|
|sum_click|해당 날짜에 학습자가 클릭한 횟수| 클릭 수 감소는 학습자의 활동 감소를 나타내며, 이탈 예측에서 중요한 역할을 함. 활동량 감소가 이탈의 전조일 수 있음.|

- 분석 방향
    - id_student별로 클릭 수(sum_click) 변화 분석.
    - 시간에 따른 활동 변화 (date)를 통해 이탈을 예측.

<br><br>

### 우선순위 2: 보조 컬럼 (보조적이지만 중요한 컬럼)
| 컬럼명| 설명| 이유|
|---------------|--------------------------------------|------------------------------------------------------|
|code_module|강좌 코드 (예: AAA)|특정 강좌에서 이탈률 차이가 있을 수 있음. 강좌별로 이탈 패턴을 분석하여, 이탈 가능성이 높은 강좌를 파악할 수 있음.|
|code_presentation|강좌 제공 학기 (예: 2013J)|특정 학기에서 이탈 패턴이 다를 수 있음. 학기별 분석을 통해 이탈 가능성이 높은 학기를 식별할 수 있음.|

- 분석 방향
    - 특정 강좌 또는 학기별로 이탈률 차이 분석. 이탈이 더 높은 강좌나 학기가 있을 수 있음.

<br><br>

### 우선순위 3: 불필요 컬럼 (이탈 예측에 크게 영향 없음)
| 컬럼명          | 설명                                   | 이유                                                   |
|---------------|--------------------------------------|------------------------------------------------------|
| id_site | 웹사이트 ID | 학습자의 웹사이트 활동과 이탈 사이에 직접적인 관계가 적을 수 있음. 이탈 예측에 큰 영향을 미치지 않음. |



In [3]:
df

,code_module,code_presentation,id_student,id_site,date,sum_click
0,AAA,2013J,28400,546652,-10,4
1,AAA,2013J,28400,546652,-10,1
2,AAA,2013J,28400,546652,-10,1
3,AAA,2013J,28400,546614,-10,11
4,AAA,2013J,28400,546714,-10,1
...,...,...,...,...,...,...
10655275,GGG,2014J,675811,896943,269,3
10655276,GGG,2014J,675578,896943,269,1
10655277,GGG,2014J,654064,896943,269,3
10655278,GGG,2014J,654064,896939,269,1


In [5]:
df = df.drop(columns=['code_module', 'code_presentation', 'id_student'])
df

,id_site,date,sum_click
0,546652,-10,4
1,546652,-10,1
2,546652,-10,1
3,546614,-10,11
4,546714,-10,1
...,...,...,...
10655275,896943,269,3
10655276,896943,269,1
10655277,896943,269,3
10655278,896939,269,1
